# MLlib hands-on: RandomForest para predecir la severidad del retraso

## Descripción de las variables

El dataset está compuesto por las siguientes variables:

1. **Year** 2008
2. **Month** 1
3. **DayofMonth** 1-31
4. **DayOfWeek** 1 (Monday) - 7 (Sunday)
5. **DepTime** hora real de salida (local, hhmm)
6. **CRSDepTime** hora prevista de salida (local, hhmm)
7. **ArrTime** hora real de llegada (local, hhmm)
8. **CRSArrTime** hora prevista de llegada (local, hhmm)
9. **UniqueCarrier** código del aparato
10. **FlightNum** número de vuelo
11. **TailNum** identificador de cola: aircraft registration, unique aircraft identifier
12. **ActualElapsedTime** tiempo real invertido en el vuelo
13. **CRSElapsedTime** en minutos
14. **AirTime** en minutos
15. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
16. **DepDelay** retraso a la salida, en minutos
17. **Origin** código IATA del aeropuerto de origen
18. **Dest** código IATA del aeropuerto de destino
19. **Distance** en millas
20. **TaxiIn** taxi in time, in minutes
21. **TaxiOut** taxi out time in minutes
22. **Cancelled** *si el vuelo fue cancelado (1 = sí, 0 = no)
23. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
24. **Diverted** *si el vuelo ha sido desviado (1 = sí, 0 = no)
25. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
26. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
27. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
28. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
29. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

**IMPORTANTE:** Vamos a emplear una versión reducida del dataset de vuelos para entender la forma de trabajar con MLlib, y en particular, los mecanimos de exploración de combinaciones de híper-parámetros para encontrar el mejor modelo, los cuales requieren ajustar varios modelos distintos varias veces. Este proceso sería demasiado largo si utilizásemos el dataset original de 300 MB. 

<p>En la siguiente celda, descargaremos este dataset reducido de Internet mediante la ejecución del comandos de Linux `wget` y lo subiremos a HDFS  con el comando de HDFS `copyFromLocal`. Los datos guardados en HDFS son volátiles y desaparecerán cuando el cluster sea desmantelado.</p>

### Vamos a utilizar una versión reducida del dataset original para poder ver en funcionamiento el ajuste de hiper-parámetros, que requiere ajustar varios modelos

#### Descargamos el CSV reducido y lo subimos a HDFS (esta vez no es Google Cloud Storage)

In [ ]:
!wget https://raw.githubusercontent.com/olbapjose/xapi-clojure/master/flights_jan08.csv
!hdfs dfs -copyFromLocal flights_jan08.csv /tmp

#### Ahora lo leemos desde la ruta /tmp/flights_jan08.csv de HDFS

In [ ]:
# Leemos el dataset de HDFS. Esta operación todavía no hace la lectura hace una pasada 
# sobre los datos para inferir el esquema
flightsDF = spark.read.option("header", "true")\
                      .option("inferSchema", "true")\
                      .csv("/tmp/flights_jan08.csv")

In [ ]:
flightsDF.printSchema()

Vamos a utilizar algunos transformadores habituales, y un algoritmo RandomForest que es un estimador. Utilizaremos:

* StringIndexer para convertir variables tipo String en variables categóricas pero cuyos valores son números reales con la parte decimal a 0, tal como necesitan los algoritmos de Spark.
* Bucketizer para discretizar la columna de ArrDelay sin dar nombre a las categorías, solo numeros. Será nuestra variable target.
* VectorAssembler para unir las columnas de las features en una sola de tipo vector
* RandomForest que es un estimador, como algoritmo de predicción de la severidad
* Pipeline, que es un estimador y que incluirá todos los elementos anteriores.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import *
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

cleanFlightsDF = flightsDF.where("ArrDelay != 'NA' and DepDelay != 'NA' and DepTime != 'NA' and ArrTime != 'NA'")\
                          .withColumn("ArrDelay", F.col("ArrDelay").cast(IntegerType()))\
                          .withColumn("DepDelay", F.col("DepDelay").cast(IntegerType()))\
                          .withColumn("ArrTime", F.col("ArrTime").cast(IntegerType()))\
                          .withColumn("DepTime", F.col("DepTime").cast(IntegerType()))

# Definimos tres categorías: <15, entre 15 y 60, >60. Cada una se codifica con un número real: 0.0, 1.0, 2.0 
splitsDelays = [-float("inf"), 15, 60, float("inf")]
arrDelayBucketizer = Bucketizer(splits=splitsDelays, inputCol="ArrDelay", outputCol="ArrDelayBucketed")

# Definimos varias franjas: 00:00 - 06:00, 06:00 - 12:00, 12:00 - 18:00, 18:00 - 22:00, 22:00 - 00:00
splitsDepTime = [-1, 600, 1200, 1800, 2200, 2500]
depTimeBucketizer = Bucketizer(splits=splitsDepTime, inputCol="DepTime", outputCol="DepTimeBucketed")

# Dividimos en train y test de manera aleatoria usando la semilla 123 para los números aleatorios, para que sea reproducible. 
# Esto devuelve una lista de dos DataFrames. La división hará que el primer elemento de la lista sea un DF 
# con aprox. el 70 % de las filas. Lo usaremos para entrenar. El otro DF tendrá aprox el 30 % restante de las filas. 
splits = cleanFlightsDF.randomSplit([0.7, 0.3], seed = 123)
trainDF = splits[0].cache() # El primer DF tiene el 70 % de los datos
testDF = splits[1].cache()

# Si quisiera generar la lista con todos los StringIndexer para TODAS las columnas:
# categoricas = ["asdñfkj", "añdslkjadf"]
# indexerList = [StringIndexer(inputCol=c, outputCol=c + "indexed", handleInvalid="skip") for c in categoricas]

# Indexamos las columnas categóricas Origin, Dest y DayOfWeek, para traducirlas a reales con la parte decimal a 0.
# Recordemos que esto también introduce metadatos adicionales en la columna resultante para indicar que, aunque sea
# una columna de números reales, en realidad están representando categorías y debe ser tratada como tal por el algoritmo
originIndexer = StringIndexer(inputCol = "Origin", outputCol="OriginIndexed", handleInvalid="skip")
destIndexer = StringIndexer(inputCol = "Dest", outputCol="DestIndexed", handleInvalid="skip")
dowIndexer = StringIndexer(inputCol = "DayOfWeek", outputCol="DayOfWeekIndexed", handleInvalid="skip")

# Ahora unimos todas las columnas que se usarán como variables en una sola columna de tipo vector llamada featuresVector
vectorAssembler = VectorAssembler(inputCols = ["DepDelay", "DepTimeBucketed", "OriginIndexed", "DestIndexed", "DayOfWeekIndexed"], 
                                  outputCol = "featuresVector")

randomForest = RandomForestClassifier(featuresCol = "featuresVector",
                                      labelCol = "ArrDelayBucketed",
                                      numTrees = 50, maxBins=100)

pipeline = Pipeline(stages=[arrDelayBucketizer, depTimeBucketizer, dowIndexer, originIndexer, destIndexer, 
                            vectorAssembler, randomForest])

Aplicamos fit para ajustar el pipeline completo. Lo que esto hace es, para cada etapa:
- Si la etapa es un Transformer, invoca al método transform() pasándole el DF que recibe de la etapa anterior,
  y envía el resultado (DF transformado) a la etapa siguiente del pipeline.
- Si la etapa es un Estimator, invoca al método fit() pasándole como argumento el DF recibido de la etapa anterior,
  y después invoca transform() sobre el objeto devuelto por fit, pasando de nuevo dicho DF como argumento. 
  Finalmente el DF devuelvo por transform es enviado a la etapa siguiente del pipeline.

In [ ]:
pipelineModel = pipeline.fit(trainDF)

Vamos a hacer predicciones sobre los datos de test. Esto devolverá un nuevo DF al que se le han añadido varias columnas al final:
1. **`rawPrediction`**: magnitud que tiene una interpretación diferente según el algoritmo pero que intuitivamente nos da una
  medida de la confianza en cada posible clase (cuanto mayor, más confianza se tiene en que esa es la clase del ejemplo). En
  nuestro caso será un vector de 3 números reales puesto que nuestro problema tiene 3 clases
2. **`probability`**: vector de probabilidades de que el ejemplo pertenezca a cada una de las 3 clases de nuestro problema
3. **`prediction`**: clase para la cual la rawProbability es mayor.

In [ ]:
predictionsDF = pipelineModel.transform(testDF)

In [ ]:
print("Hay {0} ejemplos de entrenamiento".format(trainDF.count()))

In [ ]:
predictionsDF.show()

### Evaluación del modelo (evaluamos las predicciones que hemos hecho sobre el DF de test)

In [ ]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Objeto evaluador, para evaluar las predicciones. Compara una columna de predicciones con la columna target del verdadero valor
# Hay varias métricas posibles, pero nosotros hemos elegido la más simple: porcentaje de acierto en clasificación.
evaluator = MulticlassClassificationEvaluator(
    labelCol="ArrDelayBucketed", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictionsDF)

print("Test Error = %g " % (1.0 - accuracy))

### Ajuste de híper-parámetros utilizando Cross Validation sobre el subconjunto de train

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

# Objeto ParamGrid al que le tenemos que indicar cada uno de los parámetros sobre los que queremos buscar,
# y la lista de valores posibles que queremos probar con cada uno. Deben ser parámetros tomados directamente
# del objeto estimador que se añadió al pipeline (no pueden ser de ningún otro modelo). En nuestro caso este objeto
# está almacenado en la variable randomForest que habíamos creado en celdas anteriores.
paramGrid = ParamGridBuilder()\
    .addGrid(randomForest.numTrees, [50, 100, 150])\
    .addGrid(randomForest.maxDepth, [3, 4, 5])\
    .build() # como tenemos 2 parámetros con 3 valores posibles cada uno, hay 9 combinaciones posibles

# CrossValidator es un estimador. Cuando invocamos a fit(), probará todas las combinaciones de valores de los 
# parámetros, invocando con cada combinación al método fit() del objeto pipeline que le hemos pasado como argumento
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Como hemos hecho 3 folds, habrá que entrenar 3 veces cada modelo candidato (cada combinación de parámetros)
# y obtener su media de accuracy. En total vamos a entrenar 27 modelos

cvModel = crossval.fit(trainDF)
cvModel.bestModel

#### El objeto RandomForestModel (modelo ajustado presente en la última etapa del pipeline ya ajustado) dispone de ciertos atajos para recuperar valores de algunos parámetros (por ejemplo getNumTrees) pero no de otros (por ejemplo, no existe getMaxDepth)

In [ ]:
rf = cvModel.bestModel.stages[6]
print("Número óptimo de árboles: {0}".format(rf.getNumTrees))
print("Max depth óptimo: {0}".format(rf.getOrDefault('maxDepth')))